In [33]:
import json
with open('/home/tmsincomb/Desktop/SparcFromIlx.json', 'r') as infile:
    sparc = json.load(infile)
    records = [record for entity_record in sparc for record in entity_record.values()]
    
df = pd.DataFrame(records)[['iri', 'label', 'definition']]
df
df.to_csv('/home/tmsincomb/Desktop/SparcFromIlx.csv', index=False, columns=['iri', 'label', 'definition'])

In [35]:
df.head()

,iri,label,definition
0,http://uri.interlex.org/base/ilx_0050090,Neocortex Layer II/III Martinotti cell,NaN
1,http://uri.interlex.org/base/ilx_0050008,Neocortex Neuron expressing somatostatin,NaN
2,http://uri.interlex.org/base/ilx_0050009,Neocortex Tufted pyramidal cell,NaN
3,http://uri.interlex.org/base/ilx_0050015,Neocortex Narrow pyramidal cell projecting to ...,Narrow pyramidal cells have a narrow-looking a...
4,http://uri.interlex.org/base/ilx_0050017,Neocortex Tufted pyramidal cell towards layer I,NaN


In [49]:
import nltk

w1 = set('mapping')
w2 = set('mapping is')

nltk.jaccard_distance(w1, w2)

0.25

In [87]:
%%time
from ilxutils.nltklib import sentence_similarity, get_tokenized_sentence
s1 = 'spinalis a has'
s2 = 'has a spinalis'
print(sentence_similarity(s1, s2))
print(1- nltk.jaccard_distance(set(s1), set(s2)))
print(jaccard_similarity(x=s1, y=s2))

1.0
1.0
1.0
CPU times: user 1.28 ms, sys: 0 ns, total: 1.28 ms
Wall time: 957 µs


In [53]:
from math import*
import nltk
 
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def jaccard_similarity_list(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    print(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection / union)

s1 = 'has part of'
s2 = 'has part of'
# print(jaccard_similarity_list(list1=[s2], list2=[s1]))
print(jaccard_similarity(x=s1, y=s2))
print(nltk.edit_distance(s1, s2))
print(nltk.jaccard_distance(set(s1), set(s2)))

1.0
2
0.0


In [6]:
from collections import defaultdict
# from ilxutils.ilx_pred_map import IlxPredMap
from ilxutils.interlex_sql import IlxSql
from ilxutils.mydifflib import ratio
from ilxutils.tools import open_pickle, create_pickle
from ilxutils.scicrunch_client import scicrunch
import os
import pandas as pd
from pathlib import Path 
import rdflib
from rdflib import Graph, RDF, RDFS, Literal, URIRef, OWL
from sys import exit

# ipm = IlxPredMap()
sql = IlxSql(db_url=os.environ.get('SCICRUNCH_DB_URL_PRODUCTION'), from_backup=True)
# beta_sql = IlxSql(db_url=os.environ.get('SCICRUNCH_DB_URL_BETA'))
# sci = scicrunch(api_key=os.environ.get('SCICRUNCH_API_KEY'), base_url=os.environ.get('SCICRUNCH_BASEURL_PRODUCTION'))
# beta_sci = scicrunch(api_key=os.environ.get('SCICRUNCH_API_KEY'), base_url=os.environ.get('SCICRUNCH_BASEURL_BETA'))

In [7]:
ex = sql.get_existing_ids()

In [9]:
len(ex.curie)

1625660

In [3]:
ex[ex.iri == 'http://purl.org/sig/ont/fma256700']

,tid,curie,iri,preferred,ilx,label,definition
101,50,FMA:256700,http://purl.org/sig/ont/fma256700,0,ilx_0100049,Apex of spinal cord dorsal horn (birnlex 2670),


In [4]:
fma_fix = ex[ex.iri.str.startswith('http://purl.org/sig/ont/fma') & ~ex.iri.str.startswith('http://purl.org/sig/ont/fma/')]
fma_fix.head()

,tid,curie,iri,preferred,ilx,label,definition
101,50,FMA:256700,http://purl.org/sig/ont/fma256700,0,ilx_0100049,Apex of spinal cord dorsal horn (birnlex 2670),
113,55,FMA:68597,http://purl.org/sig/ont/fma68597,0,ilx_0100054,Brodmann (1909) area 1,
117,56,FMA:68608,http://purl.org/sig/ont/fma68608,0,ilx_0100055,Brodmann (1909) area 11,
121,57,FMA:68609,http://purl.org/sig/ont/fma68609,0,ilx_0100056,Brodmann (1909) area 12,
125,58,FMA:68610,http://purl.org/sig/ont/fma68610,0,ilx_0100057,Brodmann (1909) area 13,


In [6]:
from ilxutils.remotes import interlex_remote_production as ixr
for row in fma_fix.itertuples():
    entity_update = dict(
        ilx_id = row.ilx,
        delete_existing_ids = [dict(
            iri = row.iri,
            curie = row.curie,
        )],
#         add_existing_ids = [dict(
#             iri = row.iri.replace('http://purl.org/sig/ont/fma', 'http://purl.org/sig/ont/fma/fma'),
#             curie = row.curie,
#             preferred = '0',
#         )],
    )
    print(entity_update)
    resp = ixr.update_entity(**entity_update)   
    break
resp

{'ilx_id': 'ilx_0100049', 'delete_existing_ids': [{'iri': 'http://purl.org/sig/ont/fma256700', 'curie': 'FMA:256700'}]}
[{'id': '3904010', 'tid': '50', 'curie': 'UBERON:0004678', 'iri': 'http://purl.obolibrary.org/obo/UBERON_0004678', 'curie_catalog_id': '0', 'version': '7', 'time': '1571323104', 'preferred': 1}, {'id': '3904012', 'tid': '50', 'curie': 'FMA:256700', 'iri': 'http://purl.org/sig/ont/fma/fma256700', 'curie_catalog_id': '0', 'version': '7', 'time': '1571323104', 'preferred': 0}, {'id': '3904009', 'tid': '50', 'curie': 'ILX:0100049', 'iri': 'http://uri.interlex.org/base/ilx_0100049', 'curie_catalog_id': '3', 'version': '7', 'time': '1571323104', 'preferred': 0}, {'id': '3904008', 'tid': '50', 'curie': 'BIRNLEX:2670', 'iri': 'http://uri.neuinfo.org/nif/nifstd/birnlex_2670', 'curie_catalog_id': '46', 'version': '7', 'time': '1571323104', 'preferred': 0}]


TypeError: argument of type 'NoneType' is not iterable

In [7]:
annos = sql.get_annotations()
ex = sql.get_existing_ids()
fma_annos = annos[(annos.value.str.lower().str.strip().str.startswith('fma:')) & (annos.annotation_type_label == 'hasDbXref')]
ex_fma = ex.merge(fma_annos, left_on='curie', right_on='value', how='right', suffixes=('_ex', '_annos'))
# ex_fma = ex_fma.replace(np.nan, None, regex=True)
existing_ids_to_add = []
for row in ex_fma[~ex_fma.curie.notnull()].itertuples():
    existing_ids_to_add.append(dict(
        term_ilx = row.term_ilx,
        term_tid = row.tid_annos,
        iri = f'http://purl.org/sig/ont/fma{row.value.split(":")[-1]}',
        curie = row.value,
    ))
print(existing_ids_to_add[0])

{'term_ilx': 'ilx_0100049', 'term_tid': 50, 'iri': 'http://purl.org/sig/ont/fma256700', 'curie': 'FMA:256700'}


In [ ]:
terms = sql.get_terms()

In [ ]:
terms[terms.ilx.str.isin([r['ilx'] for r in existing_ids_to_add[:2]])]

In [3]:
from ilxutils.remotes import interlex_remote_production as ixr
ixr.ilx_cli.get_entity('ilx_0100049')['existing_ids']

[{'id': '3885441',
  'tid': '50',
  'curie': 'BIRNLEX:2670',
  'iri': 'http://uri.neuinfo.org/nif/nifstd/birnlex_2670',
  'curie_catalog_id': '46',
  'version': '5',
  'time': '1571242995',
  'preferred': '0'},
 {'id': '3885442',
  'tid': '50',
  'curie': 'ILX:0100049',
  'iri': 'http://uri.interlex.org/base/ilx_0100049',
  'curie_catalog_id': '3',
  'version': '5',
  'time': '1571242996',
  'preferred': '0'},
 {'id': '3885443',
  'tid': '50',
  'curie': 'UBERON:0004678',
  'iri': 'http://purl.obolibrary.org/obo/UBERON_0004678',
  'curie_catalog_id': '0',
  'version': '5',
  'time': '1571242996',
  'preferred': '1'},
 {'id': '3885444',
  'tid': '50',
  'curie': 'FMA:256700',
  'iri': 'http://purl.org/sig/ont/fma256700',
  'curie_catalog_id': '0',
  'version': '5',
  'time': '1571242996',
  'preferred': '0'}]

In [5]:
from ilxutils.remotes import interlex_remote_test as ixrt
resp = ixrt.update_entity(
    ilx_id = 'tmp_0738406',
    add_existing_ids=[{'iri': 'http://purl.org/sig/ont/fma256703', 'curie': 'FMA:256703', 'preferred': '0'}],
)
print(resp)

[2019-10-16 09:49:33,417] -     INFO -       ontquery - interlex_client.py:829  - {'ilx_id': 'tmp_0738406', 'label': None, 'type': None, 'definition': None, 'comment': None, 'superclass': None, 'synonyms': (), 'add_existing_ids': [{'iri': 'http://purl.org/sig/ont/fma256703', 'curie': 'FMA:256703', 'preferred': '0'}], 'delete_existing_ids': None}


QueryResult({'iri': 'http://uri.interlex.org/base/tmp_0738406', 'curie': 'TMP:0738406', 'label': 'troy_test_term', 'labels': (), 'definition': 'test_0JLVM8WG9G73', 'synonyms': ('brain',), 'deprecated': None, 'predicates': {'comment': 'test_5AEAULQGEHFH'}, 'type': None, 'types': (), '_graph': None, 'source': <ontquery.plugins.services.InterLexRemote object at 0x106ea6510>})


In [13]:
ixr.update_entity(ilx_id='ilx_0100055', subThingOf='ilx_0101471')

[2019-10-16 10:30:53,400] -     INFO -       ontquery - interlex_client.py:829  - {'ilx_id': 'ilx_0100055', 'label': None, 'type': None, 'definition': None, 'comment': None, 'superclass': 'ilx_0101471', 'synonyms': (), 'add_existing_ids': None, 'delete_existing_ids': None}


QueryResult({'iri': 'http://uri.interlex.org/base/ilx_0100055', 'curie': 'ILX:0100055', 'label': 'Brodmann (1909) area 11', 'labels': (), 'definition': '', 'synonyms': ('B09-11', 'prefrontal', 'area 11 of Brodmann-1909', 'Brodmann area 11', 'Area praefrontalis'), 'deprecated': None, 'predicates': {'comment': ''}, 'type': None, 'types': (), '_graph': None, 'source': <ontquery.plugins.services.InterLexRemote object at 0x10487b9d0>})

In [15]:
for i, r in enumerate(existing_ids_to_add):
    if r['term_ilx'] == 'ilx_0725187':
        print(i)

1538


In [10]:
from ilxutils.remotes import interlex_remote_production as ixr
resps = []
start = 'ilx_0725187'
add_existing_ids.index()
for r in existing_ids_to_add[3:]:
    if r.
    resp = ixr.update_entity(
        ilx_id = r['term_ilx'],
        add_existing_ids=[{'iri': r['iri'], 'curie': r['curie'], 'preferred': '0'}],
    )
    resps.append(resp)

[2019-10-16 09:51:16,658] -     INFO -       ontquery - interlex_client.py:829  - {'ilx_id': 'ilx_0100056', 'label': None, 'type': None, 'definition': None, 'comment': None, 'superclass': None, 'synonyms': (), 'add_existing_ids': [{'iri': 'http://purl.org/sig/ont/fma68609', 'curie': 'FMA:68609', 'preferred': '0'}], 'delete_existing_ids': None}
[2019-10-16 09:51:17,786] -     INFO -       ontquery - interlex_client.py:829  - {'ilx_id': 'ilx_0100057', 'label': None, 'type': None, 'definition': None, 'comment': None, 'superclass': None, 'synonyms': (), 'add_existing_ids': [{'iri': 'http://purl.org/sig/ont/fma68610', 'curie': 'FMA:68610', 'preferred': '0'}], 'delete_existing_ids': None}
[2019-10-16 09:51:19,228] -     INFO -       ontquery - interlex_client.py:829  - {'ilx_id': 'ilx_0100058', 'label': None, 'type': None, 'definition': None, 'comment': None, 'superclass': None, 'synonyms': (), 'add_existing_ids': [{'iri': 'http://purl.org/sig/ont/fma68612', 'curie': 'FMA:68612', 'preferred'

BadResponseError: 
Error: Json could not returned
Status Code: [504]
Params: None
Url: https://scicrunch.org/api/1/term/edit/648328
Output: <html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx/1.16.1</center>
</body>
</html>


In [11]:
ex[ex.iri.isin(['http://uri.neuinfo.org/nif/nifstd/nifext_5214'])]

,tid,curie,iri,preferred,ilx,label,definition
0,1,NIFEXT:5214,http://uri.neuinfo.org/nif/nifstd/nifext_5214,1,ilx_0100000,(R)N6-phenylisopropyladenosine,


In [11]:
sub_df = sql.get_custom('select * from users')
sub_df[sub_df.email.str.contains('bern')]

,id,guid,email,password,salt,banned,level,firstName,middleInitial,lastName,organization,created,verified,verify_string,orcid_id,subscribe_email
1221,31879,32094,bernd.kuhn@roche.com,6c5f55bdd12ff399fbf378254fa18f1b,186902191955e939e1bdb1f377056168,0,0,Bernd,None,Kuhn,F. Hoffmann-La Roche,1.441348e+09,1,200205476855e939e1be1d7312922756,None,1
1381,32320,32535,bernard.maigret@loria.fr,ac816824e117a8a00b3c0d8611495860,10312377355651f7f4ed857178376806,0,0,Bernard,None,Maigret,LORIA Lorraine University,1.448212e+09,1,12731170815651f7f4edde8203179558,None,1
4832,36381,36596,bernard.de.bono@gmail.com,7ac5afa7931b57afd1a656d5b1fa9e84,4855381915c44e51bc12a1948409545,0,0,Bernard,None,de Bono,University of Auckland,1.548019e+09,1,16317758185c44e51bc14fd719809759,None,1
5012,36561,36776,bernotasgytis@yahoo.com,9e3e32fc9f884280f85f6b77a67a7434,13758225305c81573c37a1f672846617,0,0,Gytis,None,Bernotas,UWE,1.551980e+09,1,9881985175c81573c37c55872809427,0000-0002-3418-5758,1


In [3]:
terms=sql.get_terms()
print(terms.shape)
terms.head(1).to_dict()

(638299, 9)


{'tid': {0: 1},
 'ilx': {0: 'ilx_0100000'},
 'label': {0: '(R)N6-phenylisopropyladenosine'},
 'definition': {0: ''},
 'type': {0: 'term'},
 'comment': {0: ''},
 'version': {0: 6},
 'uid': {0: 32290},
 'time': {0: 1529356493}}

In [4]:
bernard = 36381
terms[terms.type == ]

,tid,ilx,label,definition,type,comment,version,uid,time
12764,12765,ilx_0112764,Axon diameter,,annotation,,2,32290,1516269676
12765,12766,ilx_0112765,Axon length,,annotation,,2,32290,1516269678
12766,12767,ilx_0112766,Axon myelination,,annotation,,2,32290,1516269680
12767,12768,ilx_0112767,Dendrite thickness,,annotation,,2,32290,1516269682
12768,12769,ilx_0112768,Number of primary dendrites,,annotation,,2,32290,1516269684
12769,12770,ilx_0112769,Overlaps some,,annotation,,2,32290,1516269686
12770,12771,ilx_0112770,Spiking features,,annotation,,2,32290,1516269689
12771,12772,ilx_0112771,Spontaneous firing patterns,,annotation,,2,32290,1516269693
15023,15024,ilx_0115023,Source,,annotation,,2,247,1516269770
15024,15025,ilx_0115024,Variable type,,annotation,,2,247,1516269773


In [17]:
hit = 0
for i, row in terms[~terms.type.str.contains('cde') & ~terms.type.str.contains('fde')].sort_values(by='time').iterrows():
    if row.uid == ber:
        hit = i
print(hit)

0


In [20]:
terms[terms.label.str.contains('max') & (terms.type == 'annotation')]

,tid,ilx,label,definition,type,comment,version
281355,304442,ilx_0381357,maxValue,,annotation,,2


In [10]:
import yaml
yaml.load('''
datatype: real
units: mm^3
min: '0'
max: 1e^6
''')

{'datatype': 'real', 'units': 'mm^3', 'min': '0', 'max': '1e^6'}

In [11]:
yaml.dump({'datatype': 'real', 'units': 'mm^3', 'min': '0', 'max': '1e^6'})

"{datatype: real, max: 1e^6, min: '0', units: mm^3}\n"

In [14]:
yaml.load("{datatype: real, max: 1e^6, min: '0', units: mm^3}")

{'datatype': 'real', 'max': '1e^6', 'min': '0', 'units': 'mm^3'}

In [2]:
beta_sci.addTerms([{'label': 'pde_test_1', 'type':'pde'}], crawl=True, _print=False)

[{'id': 305210,
  'orig_uid': 34142,
  'uid': 34142,
  'orig_cid': None,
  'cid': None,
  'ilx': 'tmp_0382128',
  'label': 'pde_test_1',
  'type': 'pde',
  'definition': None,
  'comment': None,
  'version': 1,
  'status': 0,
  'display_superclass': 1,
  'orig_time': 1549487902,
  'time': 1549487902,
  'synonyms': [],
  'superclasses': [],
  'existing_ids': [{'id': '417998',
    'tid': '305210',
    'curie': 'ILX:0382128',
    'iri': 'http://uri.interlex.org/base/ilx_0382128',
    'curie_catalog_id': '3',
    'version': '1',
    'time': '1549487902',
    'preferred': '1'}],
  'relationships': [],
  'mappings': [],
  'annotations': [],
  'ontologies': []}]

In [62]:
%%time
import rdflib 
g = rdflib.Graph().parse('example_ontology1.ttl', format='turtle')
for _ in range(1000):
    predicate = 'rdfs:label'
    reference_obj = "Retina ganglion cell".lower()
    bindings = g.query('''
        SELECT ?subj ?obj
        WHERE {
            ?subj rdfs:label '%s' .
        }
    ''' % reference_obj).bindings
bindings

CPU times: user 1.4 s, sys: 12.1 ms, total: 1.41 s
Wall time: 1.4 s


In [54]:
 %%time
import rdflib 
g = rdflib.Graph().parse('example_ontology1.ttl', format='turtle')
for _ in range(1000):
    predicate = 'rdfs:label'
    reference_obj = "Retina ganglion cell".lower()
    bindings = g.query('''
        PREFIX text: <http://jena.apache.org/text#>
        SELECT ?subj ?obj
        WHERE {
            ?subj text:query (rdfs:label 'search') ;
                 rdfs:label ?obj ;
                 FILTER (lcase(str(?obj)) = str('%s')) .
        }
    '''%reference_obj).bindings
bindings

CPU times: user 4.61 s, sys: 0 ns, total: 4.61 s
Wall time: 4.6 s


In [ ]:
g = rdflib.Graph().parse('/home/tmsincomb/Dropbox/ilx.ttl', format='turtle')

In [ ]:
%%time
for _ in range(10):
    hits = (t for t in g if isinstance(t[0], rdflib.URIRef) and 'neuron'.lower() in str(t[2]).lower())
print(len(hits))

In [71]:
%%time
[v for v in g.subject_predicates(object=rdflib.Literal('Retina ganglion cell', datatype=rdflib.XSD.string))]

CPU times: user 63 µs, sys: 0 ns, total: 63 µs
Wall time: 65.8 µs


[(rdflib.term.URIRef('http://uri.neuinfo.org/nif/nifstd/nifext_17'),
  rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#prefLabel')),
 (rdflib.term.URIRef('http://uri.neuinfo.org/nif/nifstd/nifext_17'),
  rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'))]

In [8]:
print(g.serialize(format='turtle').decode('utf-8'))

@prefix : <file:///ERROR/EMPTY/PREFIX/BANNED/> .
@prefix BFO: <http://purl.obolibrary.org/obo/BFO_> .
@prefix BIRNLEX: <http://uri.neuinfo.org/nif/nifstd/birnlex_> .
@prefix NIFEXT: <http://uri.neuinfo.org/nif/nifstd/nifext_> .
@prefix NIFRID: <http://uri.neuinfo.org/nif/nifstd/readable/> .
@prefix NLX: <http://uri.neuinfo.org/nif/nifstd/nlx_> .
@prefix NLXCELL: <http://uri.neuinfo.org/nif/nifstd/nlx_cell_> .
@prefix SAO: <http://uri.neuinfo.org/nif/nifstd/sao> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

BIRNLEX:1126 a owl:Class ;
    rdfs:label "Retina ganglion cell" ;
    rdfs:subClassOf owl:DeprecatedClass ;
    owl:deprecated true .

NIFEXT:17 a owl:Class

In [24]:
sql.get_custom("""
    select * from users where users.firstName='troy';
""")

,id,guid,email,password,salt,banned,level,firstName,middleInitial,lastName,organization,created,verified,verify_string,orcid_id,subscribe_email
0,33927,34142,troysincomb@gmail.com,7a9ff1f9cc50a6a2373358164a969edc,21388697615967ee26ee0bc943071670,0,1,troy,None,sincomb,crbs,1499983398,1,183012665967ee26ee1e7691881768,None,1


In [4]:
onto = 'https://utslogin.nlm.nih.gov/cas/login?service=https%3a%2f%2fdownload.nlm.nih.gov%2fumls%2fkss%2f2018AB%2fumls-2018AB-full.zip'
sci.addOntology(ontology_url=onto)

200


{'data': {'id': 7,
  'url': 'https://utslogin.nlm.nih.gov/cas/login?service=https%3a%2f%2fdownload.nlm.nih.gov%2fumls%2fkss%2f2018AB%2fumls-2018AB-full.zip'},
 'success': True}

In [9]:
sql.get_custom('term_curie_catalog').head()

,id,uid,prefix,namespace,name,description,homepage,logo,type,source_uri
0,1,0,,http://uri.neuinfo.org/nif/nifstd/,Neuroscience Information Framework Standard On...,None,https://neuinfo.org/,https://neuinfo.org/upload/community-logo/_863...,ontology,false
1,2,0,NLXWIKI,http://neurolex.org/wiki/,The Neuroscience Lexicon,None,http://neurolex.org/wiki/Main_Page,http://neurolex.org/w/skins/ontoskin3/img/xneu...,vocabulary,true
2,3,0,ILX,http://uri.interlex.org/base/ilx_,InterLex - The Inter-Source Inter-Vocabulary L...,None,http://scicrunch.org/browse/termdashboard,None,vocabulary,false
3,4,0,CHEBI,http://purl.obolibrary.org/obo/CHEBI_,Chemical Entities of Biological Interest,None,https://www.ebi.ac.uk/chebi/init.do,https://www.ebi.ac.uk/chebi/images/ChEBI_logo.png,ontology,true
4,5,0,GO,http://purl.obolibrary.org/obo/GO_,Gene Ontology,None,http://www.ontobee.org/,http://www.ontobee.org/public/images/logo.gif,ontology,true


In [12]:
sql.get_custom('curie_catalog')[sql.get_custom('curie_catalog').prefix == 'ILX']

,id,uid,prefix,namespace
62,95,0,ILX,http://uri.interlex.org/base/ilx_


In [5]:
annotations = sql.get_annotations_from_backup()

In [6]:
annotations.head(1).to_dict('records')[0]

{'id': 2265164,
 'tid': 1,
 'annotation_tid': 304445,
 'value': 'http://neurolex.org/wiki/nifext_5214',
 'uid': 0,
 'term_label': '(R)N6-phenylisopropyladenosine',
 'term_ilx': 'ilx_0100000',
 'term_def': '',
 'term_version': 6,
 'annotation_label': 'hasDbXref',
 'annotation_ilx': 'ilx_0381360'}

In [22]:
# cid is brocken for some reason and defaulted it to 0; ilx/search is a mad house
import requests
import json
import os

base_url = 'http://localhost:8080/api/1/'
prime_term_url = base_url + 'ilx/add'
add_term_url = base_url + 'term/add'
api_key = os.environ.get('LOCAL_SCICRUNCH_API_KEY')

def test_local_host_url():
    search_term_via_ilx_url = base_url + 'ilx/search/identifier/{ilx_id}?key={api_key}'
    search_term_via_ilx_url = search_term_via_ilx_url.format(ilx_id='tmp_0100000', api_key=api_key)
    print(search_term_via_ilx_url)
    response = requests.get(search_term_via_ilx_url, headers = {'Content-type': 'application/json'})
    print(response.status_code)
    print(response.json()['data']['ilx'])
    
test_local_host_url()

data = {
    'term': 'test_908',
    'type': 'term',
    # 'ontologies': [{'id':1}], #'url':"https://raw.githubusercontent.com/tgbugs/nlxeol/master/neurolex_basic.ttl"}],
    'key': api_key,
}

response = requests.post(prime_term_url, data=json.dumps(data), headers = {'Content-type': 'application/json'})
print(response.status_code)
print(response.json())

data.update({
    'label': data.pop('term'),
    'ilx': response.json()['data']['fragment'],
    # 'curie_catalog_id': 1,
    # 'annotations': [{'id':12345}],
})

print(data, 'pre-import')

response = requests.post(add_term_url, data=json.dumps(data), headers = {'Content-type': 'application/json'})
print(response.status_code)
try:
    print(response.json(), 'post-import')
except:
    print(response.text, 'post broke')

http://localhost:8080/api/1/ilx/search/identifier/tmp_0100000?key=ricaGK0njWn4Pru9YWIwkFdXXsGdQpj8
200
tmp_0100000
201
{'data': {'fragment': 'tmp_0149727', 'term': 'test_908', 'datetime': 1547501061, 'note': None, 'defining_url': None}, 'success': True}
{'type': 'term', 'key': 'ricaGK0njWn4Pru9YWIwkFdXXsGdQpj8', 'label': 'test_908', 'ilx': 'tmp_0149727'} pre-import
201
Incorrect integer value: '' for column 'orig_cid' at row 1Incorrect integer value: '' for column 'cid' at row 1{"data":{"id":50892,"orig_uid":34005,"uid":34005,"orig_cid":null,"cid":null,"ilx":"tmp_0149727","label":"test_908","type":"term","definition":null,"comment":null,"version":1,"status":0,"display_superclass":1,"orig_time":1547501061,"time":1547501061,"synonyms":[],"superclasses":[],"existing_ids":[{"id":"106581","tid":"50892","curie":"ILX:0149727","iri":"http:\/\/uri.interlex.org\/base\/ilx_0149727","curie_catalog_id":"3","version":"1","time":"1547501061","preferred":"1"}],"relationships":[],"mappings":[],"annotat

In [23]:
import requests
import json
import os

base_url = 'http://localhost:8080/api/1/'
update_term_url = base_url + 'term/edit/{id}'
api_key = os.environ.get('LOCAL_SCICRUNCH_API_KEY')

data = {
    'id': '50892',
    'ilx': 'tmp_0149727',
    'label': 'test_907',
    'type': 'term',
    # 'ontologies': [{'id':1}], #'url':"https://raw.githubusercontent.com/tgbugs/nlxeol/master/neurolex_basic.ttl"}],
    'key': api_key,
}
url = update_term_url.format(id=data['id'])
resp = requests.post(url, data=json.dumps(data), headers={'Content-type': 'application/json'})
try:
    print(resp.json())
except:
    print(resp.text)

Incorrect integer value: '' for column 'cid' at row 1Incorrect integer value: '' for column 'orig_cid' at row 1Incorrect integer value: '' for column 'cid' at row 1{"data":{"id":"50892","orig_uid":"34005","uid":"34005","orig_cid":"","cid":"","ilx":"tmp_0149727","label":"test_907","type":"term","definition":"","comment":"","version":2,"status":"0","display_superclass":"1","orig_time":"1547501061","time":1547501119,"synonyms":[],"superclasses":[],"existing_ids":[{"id":"106581","tid":"50892","curie":"ILX:0149727","iri":"http:\/\/uri.interlex.org\/base\/ilx_0149727","curie_catalog_id":"3","version":"1","time":"1547501061","preferred":"1"}],"relationships":[],"mappings":[],"annotations":[],"ontologies":[]},"success":true}


In [36]:
base_url = 'http://localhost:8080/api/1/'
api_key = os.environ.get('LOCAL_SCICRUNCH_API_KEY')

def addOntology(ontology_url:str) -> dict:
    add_ontology_url = base_url + 'term/ontology/add'
    data = {
        'url': ontology_url,
        'key': api_key,
    }
    response = requests.post(add_ontology_url, data=json.dumps(data), headers={'Content-type': 'application/json'})
    print(response.status_code)
    print(response.json())
    
addOntology(ontology_url='http://purl.obolibrary.org/obo/uberon/releases/2018-10-14/uberon.owl')

200
{'data': {'id': 7, 'url': 'http://purl.obolibrary.org/obo/uberon/releases/2018-10-14/uberon.owl'}, 'success': True}


In [10]:
sql.get_custom("""
    describe term_annotations;
""")

,Field,Type,Null,Key,Default,Extra
0,id,int(11) unsigned,NO,PRI,None,auto_increment
1,tid,int(11),YES,MUL,None,
2,annotation_tid,int(11),YES,MUL,None,
3,value,text,YES,,None,
4,comment,text,YES,,None,
5,upvote,int(11),YES,,None,
6,downvote,int(11),YES,,None,
7,curator_status,"enum('-1','0','+1')",NO,,0,
8,withdrawn,"enum('0','1')",NO,,0,
9,term_version,int(11),YES,,None,


In [5]:
tables.head()

,Tables_in_nif_eelg
0,_term_csv_dump
1,_term_mapping_dump
2,_term_properties
3,_term_upload_list
4,action_keys


In [11]:
beta_terms = beta_sql.get_terms()

In [12]:
beta_terms.head()

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
0,38429,,0,0,0,0,troysincomb2,def-update,,term,2,0,1,1508268793,1510959702
1,38428,ilx_0111626,0,0,0,0,troysincomb2,def-1511375706,def-update,term,30,0,1,1508267561,1512064672
2,1,tmp_0100000,32290,0,32290,0,(R)N6-phenylisopropyladenosine,,,term,7,0,1,1481226703,1516219054
3,2,tmp_0100001,32290,0,32290,0,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,,term,7,0,1,1481226705,1516219055
4,3,tmp_0100002,32290,0,32290,0,"1,2-Dibromoethane",Organic Compound;Pesticide;Gasoline Additive/C...,,term,6,0,1,1481226707,1516219057


In [2]:
terms = sql.get_terms()

In [10]:
terms[terms.id == 647862]

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
624723,647862,ilx_0724725,34142,0,34142,0,follicular antrum,Region of ovarian follicle filled with follicu...,,term,1,0,1,1542395794,1542395794


In [40]:
#{0, 1, 80, 97, 104, 105}
relationships = sql.get_relationships()

In [41]:
relationships.head()

,id,term1_tid,term1_ilx,term1_type,term2_tid,term2_ilx,term2_type,relationship_tid,relationship_ilx,relationship_label
0,1,2,ilx_0100001,term,11841,ilx_0111840,term,12785,ilx_0112784,Has role
1,2,3,ilx_0100002,term,11841,ilx_0111840,term,12785,ilx_0112784,Has role
2,3,22,ilx_0100021,term,3721,ilx_0103720,term,12785,ilx_0112784,Has role
3,4,25,ilx_0100024,term,3577,ilx_0103576,term,12785,ilx_0112784,Has role
4,5,27,ilx_0100026,term,6459,ilx_0106458,term,12797,ilx_0112796,Related to


In [7]:
from ontquery.plugins.interlex_client import InterLexClient
beta_sci = InterLexClient(
    api_key = os.environ.get('INTERLEX_API_KEY'),
    base_url = 'https://beta.scicrunch.org/api/1/',
)

In [20]:
beta_sci.get_relationship_via_tid('1432')

SystemExit: {'errormsg': 'Could not complete request', 'success': False}: with status code [404] and params:{'errormsg': 'Could not complete request', 'success': False}

/home/tmsincomb/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [36]:
api_key = os.environ.get('INTERLEX_API_KEY', 'SCICRUNCH_API_KEY')
api_key

'Mf8f6S5gD6cy6cuRWMyjUeLGxUPfo0SZ'

In [15]:
%time terms = sql.get_terms()

CPU times: user 6.51 s, sys: 560 ms, total: 7.07 s
Wall time: 22 s


In [14]:
%time annos = sql.get_annotations()

CPU times: user 24.9 s, sys: 2.78 s, total: 27.6 s
Wall time: 1min


In [23]:
%time synonyms = sql.get_synonyms()

CPU times: user 2.51 s, sys: 276 ms, total: 2.78 s
Wall time: 3.25 s


In [24]:
%time superclasses = sql.get_superclasses()

CPU times: user 2.86 s, sys: 299 ms, total: 3.16 s
Wall time: 3.59 s


In [25]:
%time relationships = sql.get_relationships()

CPU times: user 130 ms, sys: 12.1 ms, total: 142 ms
Wall time: 178 ms


In [47]:
%time existing_ids = sql.get_existing_ids()

CPU times: user 8.21 s, sys: 1.1 s, total: 9.32 s
Wall time: 18.9 s


#################################################################################################################

In [3]:
terms.head()

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
0,1,ilx_0100000,32290,0,32290,0,(R)N6-phenylisopropyladenosine,,,term,6,0,1,1481226703,1529356493
1,2,ilx_0100001,32290,0,32290,0,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,,term,6,0,1,1481226705,1529356493
2,3,ilx_0100002,32290,0,34142,0,"1,2-Dibromoethane",Organic Compound;Pesticide;Gasoline Additive/C...,,term,8,0,1,1481226707,1535122210
3,4,ilx_0100003,32290,0,32290,0,1-120 Truncated Alpha-Synuclein Drosophila,This transgenic drosophila expresses a form of...,,term,6,0,1,1481226709,1529356493
4,5,ilx_0100004,32290,0,32290,0,1-D extent,A quality inhering in a bearer by virtue of it...,,term,5,0,1,1481226711,1529356493


In [6]:
annos.head()

,id,tid,annotation_tid,value,uid,term_label,term_ilx,annotation_label,annotation_ilx
0,2265164,1,304445,http://neurolex.org/wiki/nifext_5214,0,(R)N6-phenylisopropyladenosine,ilx_0100000,hasDbXref,ilx_0381360
1,2265173,2,304445,http://neurolex.org/wiki/T3D0044,0,"1,2-Dibromo-3-chloropropane",ilx_0100001,hasDbXref,ilx_0381360
2,2265170,3,304445,http://neurolex.org/wiki/T3D0035,0,"1,2-Dibromoethane",ilx_0100002,hasDbXref,ilx_0381360
3,2265171,4,304445,http://neurolex.org/wiki/nlx_organ_090801,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,hasDbXref,ilx_0381360
4,2278612,4,304535,PMID:17376994,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,literatureCitation,ilx_0381449


In [108]:
synonyms.head()

,id,tid,literal,ilx,label
0,31340,15070,Age,ilx_0115067,Participant Age
1,31341,15070,Patient Age,ilx_0115067,Participant Age
2,31342,15070,Subject Age,ilx_0115067,Participant Age
23085,331422,3729,eletriptan,ilx_0103728,Eletriptan
23086,333931,4511,GABA-A RECEPTOR,ilx_0104510,GABRA1


In [4]:
superclasses.head()

,id,tid,superclass_tid,term_label,term_ilx,superclass_label,superclass_ilx
0,12494,15070,15069,Participant Age,ilx_0115067,National Cancer Institute Common Data Element,ilx_0115066
23078,307260,2483,10842,Connected spatiotemporal region,ilx_0102482,Spatiotemporal region,ilx_0110841
23079,307261,2484,11600,Connected temporal region,ilx_0102483,Temporal region,ilx_0111599
23080,307331,2558,10743,Cord blood stem cell,ilx_0102557,Somatic stem cell,ilx_0110742
23081,307856,3092,2515,Dependent continuant,ilx_0103091,Continuant,ilx_0102514


In [7]:
relationships.head()

,id,term1_tid,term1_ilx,term1_type,term2_tid,term2_ilx,term2_type,relationship_tid,relationship_ilx,relationship_label
0,1,2,ilx_0100001,term,11841,ilx_0111840,term,12785,ilx_0112784,Has role
1,2,3,ilx_0100002,term,11841,ilx_0111840,term,12785,ilx_0112784,Has role
2,3,22,ilx_0100021,term,3721,ilx_0103720,term,12785,ilx_0112784,Has role
3,4,25,ilx_0100024,term,3577,ilx_0103576,term,12785,ilx_0112784,Has role
4,5,27,ilx_0100026,term,6459,ilx_0106458,term,12797,ilx_0112796,Related to


In [48]:
existing_ids.head()

,id,tid,curie,iri,preferred,ilx,type,label,definition,comment
0,2181964,1,NIFEXT:5214,http://uri.neuinfo.org/nif/nifstd/nifext_5214,1,ilx_0100000,term,(R)N6-phenylisopropyladenosine,,
1,2181981,1,ILX:0100000,http://uri.interlex.org/base/ilx_0100000,0,ilx_0100000,term,(R)N6-phenylisopropyladenosine,,
2,2182003,2,T3D:0044,http://t3db.org/toxins/T3D0044,0,ilx_0100001,term,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,
3,2182004,2,ILX:0100001,http://uri.interlex.org/base/ilx_0100001,1,ilx_0100001,term,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,
4,2744315,3,T3D:0035,http://t3db.org/toxins/T3D0035,0,ilx_0100002,term,"1,2-Dibromoethane",Organic Compound;Pesticide;Gasoline Additive/C...,


#################################################################################################################

In [5]:
'#'*100 ### microload

'####################################################################################################'

In [ ]:
# 'GO:0005874'
# TODO: update sql to ignore depricated terms aka annotation -> deprecated True OPTIONAL relationship -> replacedBy <other ilx_id> & obsReason <merged IAO>
# TODO: create deprecateTerm(ilx_ids, replaced=False, replaced_reason='merged')

In [10]:
data = [{
    'label': 'isTreatedAsSynonymOf',
    'type': 'relationship',
    'comment': 'Multiple types of synonyms involved.',
}]

In [60]:
for term_data in sci.ilxSearches(['ILX:0109469', 'ILX:0109470'], crawl=True, _print=False).values():
    print('{')
    print("    'id': '" + term_data['id'] + "',")
    print("    'ilx': '" + term_data['ilx'] + "',")
    #print("    'existing_ids': {'curie': '', 'iri': '', 'delete': True},")
    print('},')
    print('LABEL |', term_data['label'])
    print('DEF   |', term_data['definition'])
    print('EX_IDs|', ["'existing_ids': {'curie': '" + d['curie'] + "', 'iri': '" + d['iri'] + "', 'delete': True}," for d in term_data['existing_ids']])
    print()

{
    'id': '9470',
    'ilx': 'ilx_0109469',
},
LABEL | Protostomia
DEF   | 
EX_IDs| ["'existing_ids': {'curie': 'BIRNLEX:534', 'iri': 'http://uri.neuinfo.org/nif/nifstd/birnlex_534', 'delete': True},", "'existing_ids': {'curie': 'ILX:0109469', 'iri': 'http://uri.interlex.org/base/ilx_0109469', 'delete': True},", "'existing_ids': {'curie': 'NCBITaxon:33317', 'iri': 'http://purl.obolibrary.org/obo/NCBITaxon_33317', 'delete': True},"]

{
    'id': '9471',
    'ilx': 'ilx_0109470',
},
LABEL | Protostomia (birnlex 334)
DEF   | 
EX_IDs| ["'existing_ids': {'curie': 'BIRNLEX:334', 'iri': 'http://uri.neuinfo.org/nif/nifstd/birnlex_334', 'delete': True},", "'existing_ids': {'curie': 'ILX:0109470', 'iri': 'http://uri.interlex.org/base/ilx_0109470', 'delete': True},", "'existing_ids': {'curie': 'NCBITaxon:33317', 'iri': 'http://purl.obolibrary.org/obo/NCBITaxon_33317', 'delete': True},"]



In [61]:
special_term_updates = [
    {
        'id': '12049',
        'ilx': 'ilx_0112048',
        'existing_ids': {'curie': 'UBERON:0002434', 'iri': 'http://purl.obolibrary.org/obo/UBERON_0002434', 'delete': True},
    },
    {
        'id': '880',
        'ilx': 'ilx_0100879',
        'label': 'Arctiini',
        'existing_ids': {'curie': 'NCBITaxon:30225', 'iri': 'http://purl.obolibrary.org/obo/NCBITaxon_30225', 'delete': True},
    },
    {
        'id': '880',
        'ilx': 'ilx_0100879',
        'existing_ids': {'curie': 'NCBITaxon:132200', 'iri': 'http://purl.obolibrary.org/obo/NCBITaxon_132200'},
    },
    {
        'id': '9470',
        'ilx': 'ilx_0109469',
        'existing_ids': {'curie': 'NCBITaxon:33317', 'iri': 'http://purl.obolibrary.org/obo/NCBITaxon_33317', 'delete': True},
    },
]
special_dep_additions = [
    {
       'ilx_id': 'ilx_0109469',
       'note': 'Duplicate of ilx_0109470 with wrong information',
    },
]
special_editor_notes = [
    {
        'ilx_id': 'ilx_0109469',
        'note': "BIRNLEX:534 is not possible with it being a subclass of an entity when it is really a superclass of that entity",
    },

]
special_merge_relationship_additions = [
    {
       'ilx1_id': 'ilx_0109469',
       'ilx2_id': 'ilx_0109470',
    },
]

[sci.updateTerms([v], crawl=True, _print=False) for v in special_term_updates]
[sci.deprecate_entity(**v) for v in special_dep_additions]
[sci.add_merge_properties(**v) for v in special_merge_relationship_additions]
[sci.add_editor_note(**v) for v in special_editor_notes]

[{'tid': '9470', 'annotation_tid': '306375', 'value': 'True', 'term_version': '5', 'annotation_term_version': '1', 'batch-elastic': 'True'}, {'tid': '9470', 'annotation_tid': '306378', 'value': 'Duplicate of ilx_0109470 with wrong information', 'term_version': '5', 'annotation_term_version': '1', 'batch-elastic': 'True'}]
{'term1_id': '9470', 'term2_id': '9471', 'relationship_tid': '306376', 'term1_version': '5', 'term2_version': '4', 'relationship_term_version': '1'}
{'tid': '9470', 'annotation_tid': '306377', 'value': 'http://purl.obolibrary.org/obo/IAO_0000227', 'term_version': '5', 'annotation_term_version': '1', 'batch-elastic': 'True'}
{'tid': '9470', 'annotation_tid': '306378', 'value': 'BIRNLEX:534 is not possible with it being a subclass of an entity when it is really a superclass of that entity', 'term_version': '5', 'annotation_term_version': '1', 'batch-elastic': 'True'}


[None]

In [6]:
#sci.addTerms(data, crawl=True, _print=False)

In [4]:
'#'*100 ### LABEL DIFF WITH MESH 

'####################################################################################################'

In [3]:
ilx = open_pickle('/home/tmsincomb/Dropbox/ilx.pickle')

In [4]:
mesh = open_pickle('/home/tmsincomb/Dropbox/scidumps/umls/mesh/mesh2018.pickle')

In [28]:
type(mesh)

rdflib.graph.Graph

In [ ]:
from rdflib import Graph, RDF, URIRef, Literal

In [5]:
%%time
g = Graph()
ilx_objs = {str(o).lower().strip():True for s,p,o in ilx.triples((None, None, None))}
#mesh_objs = {o for s,p,o in ilx.triples((None, None, None))}
for subj, pred, obj in mesh.triples((None, RDF.type, None)):
    for p, o in mesh.predicate_objects(subject=subj):
        #if set(str(o).lower().strip()) & ilx_objs:
        if ilx_objs.get(str(o).lower().strip()):
            g.add((subj, pred, obj))
            g.add((subj, p, o))
            
g.serialize('/home/tmsincomb/Desktop/same.ttl', format='turtle')

CPU times: user 2min 7s, sys: 227 ms, total: 2min 7s
Wall time: 2min 7s


In [5]:
#g = Graph()
#g.parse('/home/tmsincomb/Desktop/same.ttl', format='turtle')
g = open_pickle(Path.home()/'Desktop/same.pickle')

In [8]:
preds = {p for s,p,o in g.triples((None, None, None))}
for pred in preds:
    prefix, namespace, name = g.compute_qname(pred)
    #print(prefix, name)
preds

{rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#abbreviation'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#altLabel'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#annotation'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#casn1_label'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#entryVersion'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#frequency'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#historyNote'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#lexicalTag'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#nlmClassificationNumber'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#note'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#onlineNote'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#prefLabel'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#previousIndexing'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#publicMeSHNote'),
 rdflib.term.URIRef('http://i

In [6]:
ilx_label2row = sql.get_label2row()

In [7]:
from ilxutils.mydifflib import ratio

In [16]:
%%time
### LABEL
#ilx_objs = {str(o).lower().strip():True for s,p,o in ilx.triples((None, None, None))}
lquery = """
    select ?subj ?label
    where {
        ?subj rdf:type ns1:Concept ; 
              rdfs:label ?label ;
    } 
"""
dquery = """
    select ?definition
    where {
        %s rdf:type ns1:Concept ; 
           ns1:scopeNote ?definition ;
    } 
""" 
label_count = 0
def_count = 0
common_labels = []
data = []

for binding in g.query(lquery):
    subj, label = binding
    ilx_rows = ilx_label2row.get(str(label).lower().strip())
    
    if not ilx_rows: continue
    label_count += 1
    
    bindings = mesh.query(dquery % ("<"+str(subj)+">")).bindings
    if not bindings: continue
    def_count += 1
    definition = str(bindings[0][rdflib.term.Variable('definition')])

    for row in ilx_rows:
        hit = False
        _ratio = 0
        if definition and row['definition']:
            _ratio = ratio(definition.lower().strip(), row['definition'].lower().strip())
            if _ratio > .7:
                hit = True
                
        data.append({
            'accept':'',
            'equal':hit,
            'def_ratio':_ratio,
            'ilxdef':row['definition'],
            'meshdef':definition,
            'ilxlabel':row['label'],
            'meshlabel':label,
        })

print('Labels found:', label_count)
print('Labels w/def:', def_count)

Labels found: 2231
Labels w/def: 1501
CPU times: user 7.04 s, sys: 4.08 ms, total: 7.05 s
Wall time: 7.04 s


In [17]:
cols = ['accept', 'equal', 'def_ratio', 'ilxdef', 'meshdef', 'ilxlabel', 'meshlabel']

In [18]:
diff = pd.DataFrame(data, columns=cols)